## Logistic calssification (Binary classification)

1. 특징
    - 둘중에 어디에 속하는지 분류하는 모형이다.
    - 분류 알고리즘 중에 정확도가 가장 높은 알고리즘이다.
    - Machine Learning이 0과 1 사이의 값으로 예측한다.
    - 이후 0.5 이상이면 1, 0.5 미만이면 0으로 간주한다.
    - Machine Learing의 '근간'이 되는 모델이다.



2. H 함수 (Hypothesis function)
    - 이전의 직선 함수는 x값이 커지면 커질수록 직선의 기울기가 기울어져 버린다는 단점이 있다.
    - 0, 1을 잘 분류할수 있는 함수를 찾았고, 이는 곡선의 형태가 되었다.
    - 경계점에서 기울어지는 함수를 찾았다.
    - 그 함수는 이전의 liner regresion 함수인 H를 시그모이드로 취하면 만들어 지는 함수이다.
    - 함수 : 1 / ( 1 + e ^ (-H) )
    
    __H 함수는 해결이 되었지만, Cost 함수에 Local minimal이 생기는 문제가 생겼다.__        
    
        
3. Cost 함수 (Cost function)
    - Gradient descent(최소 제곱법)을 쓸 수 없다.
    - cost 함수에 Local minimal 이 존재 할 시, Global minimal을 구하는 데 문제가 생긴다.
    - Gridient descent(최소 제곱법)는 Cost 함수가 convex 함수 일 때에만 쓸 수 있따.
        - convex 함수 : 어디에서 시작하더라도 최소값을 구할 수 있는 함수.    
    - ___이 Local minimum을 없애기 위해 기조의 cost함수에 log를 취한다.___ -> 쭉 펴진다.

In [2]:
## Logistic Regression ( binary classification)
import tensorflow as tf

In [28]:
## 1. training data set, test data set
# Logistic classification부터는 Trainging과 Test 데이터 셋을 나누어야 한다.
# Why? 
# 실무에 많이 쓰이기 때문에 성능이 중요하고, 이에 따라 트레이닝 셋과 별개로 테스트 데이터를 통한 테스트를 통해 성능을 테스트한다.
# Test 데이터 셋은 Traing 하는 데 사용되면 안된다.
# 보통 7:3, 8:2 비율로 training data와 test data를 분리해서 사용한다.

### 하지만! 이번에는 학습의 목표는 Logistic Regression의 원리를 아는 것이기에 따로 나누지 않고 같은 데이터를 사용하겠다. ###

x_data = [[10, 0],
         [8,1],
         [3,3],
         [2,3],
         [5,1],
         [2,0],
         [1,0]]
y_data = [[1], [1], [1], [1], [0], [0], [0]]


## 2. Place Holder
X = tf.placeholder(shape=[None, 2], dtype = tf.float32) # 입력값은 n개의 학습 데이터가 들어오기에 None, Y값은 2열이기에 2이다.
Y = tf.placeholder(shape=[None, 1], dtype = tf.float32) # 입력값은 n개의 결과값이 들어오기에 None, Y값은 1행이기에 1이다.


## 3. Weight & bias
W = tf.Variable(tf.random_normal([2,1]), name = "weight")
b = tf.Variable(tf.random_normal([1]), name = "bias")


## 4. Hypothesis
logit = tf.matmul(X, W) + b  # cost에서 쓰기에 logit에 담아준다.
H = tf.sigmoid(logit) #  Linear regression에서 사용되었던 H-function을 Sigmoid 함수(곡선의 식)로 만들어 준다.


## 5. cost functoin (loss fucntion)
# cost= tf.reduce_mean(tf.square(H-Y)) # 이건 기존의 Linear regression의 cost fucntoin이다.
cost = tf.reduce_mean(tf.nn.sigmoid_cross_entropy_with_logits(logits = logit, labels=Y))
# 위의 함수가 기존의 Linear regression함수의 Local minimal을 제거해준다. 기존의 함수와 비교해보자.


## 6. train
train = tf.train.GradientDescentOptimizer(learning_rate=0.01).minimize(cost)


## 7. session & 초기화
sess = tf.Session()
sess.run(tf.global_variables_initializer())


## 8. training 학습실행
for step in range(3001):
    tmp_train, cost_val = sess.run([train, cost], feed_dict = {X:x_data, Y:y_data})
    
    if step % 300 == 0 :
        print("step : ", step,"     cost : ", cost_val)


step :  0      cost :  0.99704045
step :  300      cost :  0.71173877
step :  600      cost :  0.5765404
step :  900      cost :  0.47728434
step :  1200      cost :  0.40812802
step :  1500      cost :  0.36348096
step :  1800      cost :  0.33577412
step :  2100      cost :  0.3173408
step :  2400      cost :  0.3034835
step :  2700      cost :  0.2921496
step :  3000      cost :  0.28249034


In [12]:
## 9. accuracy (정확도 측정)
# python에서 True, False => 1, 0으로 변환가능

predict = tf.cast(H >= 0.5, dtype=tf.float32) # H는 nupy vector이다. (여러개의 값) Numpy는 행렬값에 맞게 0.5를 늘려준다.
# H는 [0.5, 0.7, 0.3, ...] 이런식으로 나올 것이다. 그럼 H>=0.5 값은 [ 1, 1, 0. ...] 이런식으로 나올 것이다.
# 그걸 다시 tf.cast함수를 통해 float형식(실수)으로 바꾸어준다. [ 1.0, 1.0, 0.0, ...]

correct = tf.equal(predict, Y) # predic 결과값이 Y(실제 결과) 와 같은지 비교한느 함수이다.
# 위 함수는 [True, False, True, ... ] 이런식으로 결과값이 나온다.
# 이 값들을 실수로 바꾼 뒤에, 평균을 내면 정확도가 몇인지 나온다.

accuracy = tf.reduce_mean(tf.cast(correct, dtype=tf.float32)) # reduce_mean : 평균내는 함수, cast : 형변환 함수


#위의 코드들은 모두 정의부이다. 이제 실제로 실행해서 accuracy를 구해야한다.
print("정확도 : {}".format(sess.run(accuracy, feed_dict={X:x_data, Y:y_data}))) # 여기에서는 training값을 test에도 이용했다.

정확도 : 0.8571428656578064


이렇게 모델이 완성되었다.
이제 완성된 모델로 predict를 해보자.

In [26]:
## 10. predict
print("x1이 3, x2가 2인 경우, Y값이 1일 확률은 몇인가 : {}".format(sess.run(H, feed_dict={X:[[3, 2]]})))

# 이걸 0.5를 기준으로 분류하면 0 or 1로 분류할 수 있다.

x1이 3, x2가 2인 경우, Y값이 1일 확률은 몇인가 : [[0.7586075]]


- - - 
이후 Logistic regression을 연습하기 위해서는 대표적인 예제인 Titinic 예제를 통해서 하는 것이 좋다.    
이와 관련된 정보는 Kaggle에 있다.